In [6]:
import pandas as pd


ds = pd.read_csv("train.csv")

ds["Age"] = ds["Age"].fillna(ds["Age"].median())


ds.loc[ds["Sex"] == "male","Sex"] = 0
ds.loc[ds["Sex"] == "female", "Sex"] = 1

ds["Embarked"] = ds["Embarked"].fillna("S")
ds.loc[ds["Embarked"] == 'S', "Embarked"] = 0
ds.loc[ds["Embarked"] == 'C', "Embarked"] = 1
ds.loc[ds["Embarked"] == 'Q', "Embarked"] = 2

from sklearn.linear_model import LinearRegression

from sklearn.cross_validation import KFold

predictors = ["Pclass", "Sex", "Age", "SibSp","Parch","Fare", "Embarked"]
alg = LinearRegression()
kf = KFold(ds.shape[0], n_folds=3, random_state=1)
predictions = []

for train, test in kf:
    # The predictors we're using the train the algorithm.  Note how we only take the rows in the train folds.
    train_predictors = (ds[predictors].iloc[train,:])
    # The target we're using to train the algorithm.
    train_target = ds["Survived"].iloc[train]
    # Training the algorithm using the predictors and target.
    alg.fit(train_predictors, train_target)
    # We can now make predictions on the test fold
    test_predictions = alg.predict(ds[predictors].iloc[test,:])
    predictions.append(test_predictions)

import numpy as np
predictions = np.concatenate(predictions, axis = 0)

predictions[predictions >  .5 ] = 1
predictions[predictions <= .5 ] = 0

values = ds["Survived"] == predictions 
accuracy = values[values == True].size / values.size
print(accuracy)

from sklearn import cross_validation
from sklearn.linear_model import LogisticRegression
alg = LogisticRegression(random_state=1)
scores = cross_validation.cross_val_score(alg, ds[predictors], ds["Survived"], cv=3)
print(scores.mean())

0.7833894500561167
0.787878787879


C:\Users\murganoo\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\base.py:175: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  args, varargs, kw, default = inspect.getargspec(init)
C:\Users\murganoo\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\base.py:175: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  args, varargs, kw, default = inspect.getargspec(init)
C:\Users\murganoo\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\base.py:175: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  args, varargs, kw, default = inspect.getargspec(init)
C:\Users\murganoo\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\base.py:175: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  args, varargs, kw, default = inspect.getargspec(init)
C:\Users\murganoo\AppData\Local\Continuum\Anaconda3\lib\site

In [119]:
ds_test = pd.read_csv("test.csv")
ds_test["Age"] = ds_test["Age"].fillna(ds["Age"].median())
ds_test.loc[ds_test["Sex"] == "male","Sex"] = 0
ds_test.loc[ds_test["Sex"] == "female","Sex"] = 1
ds_test["Embarked"] = ds_test["Embarked"].fillna("S")
ds_test.loc[ds_test["Embarked"] == "S", "Embarked"] = 0
ds_test.loc[ds_test["Embarked"] == "C", "Embarked"] = 1
ds_test.loc[ds_test["Embarked"] == "Q", "Embarked"] = 2
ds_test["Fare"] = ds_test["Fare"].fillna(ds_test["Fare"].median())

alg = LogisticRegression(random_state = 1)
alg.fit(ds[predictors], ds["Survived"])
predictions = alg.predict(ds_test[predictors])

submission = pd.DataFrame({
        "PassengerId" : ds_test["PassengerId"],
        "Survived" : predictions
    })
submission.to_csv("Kaggle.csv", index=False)